<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2

import warnings
warnings.filterwarnings('ignore')

In [2]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''
select
    count(id) cnt_vacancies
from public.vacancies
'''


In [5]:
# результат запроса
cnt_vacancies = pd.read_sql(query_3_1, connection)
cnt_vacancies

,cnt_vacancies
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''
select
    count(id) cnt_employers
from public.employers
'''

In [7]:
# результат запроса
cnt_employers = pd.read_sql_query(query_3_2, connection)
cnt_employers

,cnt_employers
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''
select
    count(id) cnt_areas
from public.areas
'''

In [9]:
# резульcnt_areasтат запроса
cnt_areas = pd.read_sql_query(query_3_3, connection)
cnt_areas

,cnt_areas
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''
select
    count(id) cnt_industries
from public.industries
'''

In [11]:
# результат запроса
cnt_industries = pd.read_sql_query(query_3_4, connection)
cnt_industries

,cnt_industries
0,294


***

### Вывод:
В базе содержится 49197 вакансий, 23501 работодателей из 1362 Регионов и охватывают 294 сферы деятельности

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# текст запроса
query_4_1 = f'''
select
    a.name "Название города",
    count(v.id) "Количество вакансий"
from public.areas a
join public.vacancies v on a.id=v.area_id
group by a.name
order by 2 desc

'''

In [13]:
# результат запроса
cnt_vacancies_in_areas = pd.read_sql_query(query_4_1, connection)
cnt_vacancies_in_areas.head(5)

,Название города,Количество вакансий
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# текст запроса
query_4_2 = f'''
select
    count(v.id) "Количество вакансий с зп"
from public.vacancies v 
where (v.salary_from is not null) or (v.salary_to is not null)
'''

In [15]:
# результат запроса
cnt_areas_with_salary = pd.read_sql_query(query_4_2, connection)
cnt_areas_with_salary

,Количество вакансий с зп
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# текст запроса
query_4_3 = f'''
select
    round(avg(v.salary_from)) "среднее значение нижней границы зп",
    round(avg(v.salary_to)) "среднее значение верхней границы зп"
from public.vacancies v
'''

In [17]:
# результат запроса
avg_min_max_salary = pd.read_sql_query(query_4_3, connection)
avg_min_max_salary

,среднее значение нижней границы з,среднее значение верхней границы
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = f''' 
select 
    v.schedule "тип рабочего графика",
    v.employment "тип трудоустройства",
    count(v.id) "количество вакансий"
from public.vacancies v 
group by 1, 2
order by 3 desc
'''

In [ ]:
# результат запроса
cnt_vac_sch_ampl = pd.read_sql_query(query_4_4, connection)
cnt_vac_sch_ampl

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [ ]:
# текст запроса
query_4_5 = f''' 
select 
    v.experience
from public.vacancies v 
group by 1
order by count(v.id)
'''

In [ ]:
# результат запроса
experience_by_cnt_vac = pd.read_sql_query(query_4_5, connection)
experience_by_cnt_vac

,experience
0,Более 6 лет
1,Нет опыта
2,От 3 до 6 лет
3,От 1 года до 3 лет


***

### Выводы:
В пятерку лидеров-городов по количеству вакансий входят Москва, Санкт-Питербург, Минск, Новосибирск, Алмааты. Это самые крупные региональные центры СНГ.

24073 вакансии имеют верхнюю или нижнюю границу зарплаты. Это означает, что в половине вакансий не указана зарплата

Средняя нижняя граница равна 71065 руб, а средняя верхняя 110537 руб.

Тип рабочего графика Удаленная работа с полной занятостью стал приемлем для работодателя и вышел на второе место по популярности.

Максимально запрашиваемы опыт работы от 1 года до 3 лет.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
# текст запроса
query_5_1 = f''' 
select 
    e.name "Работодатель",
    count(v.id) "Количество вакансий"
from public.vacancies v 
join public.employers e on v.employer_id=e.id
group by 1
order by count(v.id) desc
limit 5
'''

In [ ]:
# результат запроса
employers_by_cnt_vac = pd.read_sql_query(query_5_1, connection)
display('На первом месте', employers_by_cnt_vac.iloc [0], 'На пятом месте', employers_by_cnt_vac.iloc [4] )

'На первом месте'

Работодатель           Яндекс
Количество вакансий      1933
Name: 0, dtype: object

'На пятом месте'

Работодатель           Газпром нефть
Количество вакансий              331
Name: 4, dtype: object

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
# текст запроса
query_5_2 = f''' 
select 
    a.name "Регион",
    count(e.id) "Количество работодателей",
    count(v.id) "Количество вакансий"
from public.areas a  
left join public.vacancies v on a.id=v.area_id
left join public.employers e on e.area=a.id
group by a.name
order by count(v.id) , count(e.id) desc
'''

In [ ]:
# результат запроса
empl_and_vac_at_areas = pd.read_sql_query(query_5_2, connection)
display('Регион с большим числом работодателей и без вакансий', empl_and_vac_at_areas.iloc[0])

'Регион с большим числом работодателей и без вакансий'

Регион                      Россия
Количество работодателей       410
Количество вакансий              0
Name: 0, dtype: object

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
# текст запроса
query_5_3 = f''' 
select 
    e.name "Работодатель",
    count(distinct v.area_id) "Количество регионов"
from public.employers e  
join public.vacancies v on e.id=v.employer_id
group by 1
order by 2 desc
'''

In [ ]:
# результат запроса
empl_and_cnt_areas = pd.read_sql_query(query_5_3, connection)
display('максимальное число регионов для вакансий от работодателя', empl_and_cnt_areas.iloc[0])

'максимальное число регионов для вакансий от работодателя'

Работодатель           Яндекс
Количество регионов       181
Name: 0, dtype: object

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [ ]:
# текст запроса
query_5_4 = f''' 
select 
    count(e.id)
from public.employers e  
    left join public.employers_industries ei on e.id=ei.employer_id
where ei.industry_id is null
'''

In [ ]:
# результат запроса
empl_null_industry = pd.read_sql_query(query_5_4, connection)
display('работодателей, у которых не указана сфера деятельности', empl_null_industry.iloc[0][0])

'работодателей, у которых не указана сфера деятельности'

8419

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [ ]:
# текст запроса
query_5_5 = f''' 
select 
    e.name "Работодатель"
from public.employers e  
    left join public.employers_industries ei on e.id=ei.employer_id
group by e.name
having count(distinct ei.industry_id)= 4
order by e.name
offset 2 limit 1
'''

In [ ]:
# результат запроса
empl_with_4ind_3 = pd.read_sql_query(query_5_5, connection)
display('компания на третьем месте, у которой указано четыре сферы деятельности', empl_with_4ind_3.iloc[0][0])

'компания на третьем месте, у которой указано четыре сферы деятельности'

'2ГИС'

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [ ]:
# текст запроса
query_5_6 = f''' 
select 
    count(ei.employer_id)
from public.employers_industries ei  
    join public.industries i on i.id=ei.industry_id
where i.name = 'Разработка программного обеспечения'
'''

In [ ]:
# результат запроса
cnt_employers_IT = pd.read_sql_query(query_5_6, connection)
display('работодателей в качестве сферы деятельности указана «Разработка программного обеспечения»', cnt_employers_IT.iloc[0][0])

'работодателей в качестве сферы деятельности указана «Разработка программного обеспечения»'

3553

***

### Выводы:
С огромным отрывом по количеству вакансий и количеству регионов, в которых публикуются вакансии первенство занимает компания Яндекс. Что говорит о большой потребности в IT-специалистах.

Так же лидирующие позиции по количеству вакансий (более 100) и по количеству регионов занимают компании – это IT компании, банки и компании занимающиеся добычей полезных ископаемых. Что говорит о представительствах во многих регионах, возможностью удаленной работы или большой текучке, в добывающих компаниях – вахтовый метод.

Стоит отметить, что много (около 20%) компаний не указали сферу деятельности, поэтому полагаться на эти данные нужно с вниманием. Но даже с этим фактом можно говорить, что как минимум 7% компаний указали «Разработку программного обеспечения» как сферу детельности.


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса
query_6_1 = f''' 
select 
    count(v.id)
from public.vacancies v  
where (lower(v.name) like '%data%') or (lower(v.name) like '%данн%') 
'''

In [ ]:
# результат запроса
cnt_data_vac = pd.read_sql_query(query_6_1, connection)
display('Вакансий, имеющих отношение к данным:', cnt_data_vac.iloc[0][0])

'Вакансий, имеющих отношение к данным:'

1771

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# текст запроса
query_6_2 = f''' 
select 
    count(v.id)
from public.vacancies v  
where (
    (lower(v.name) like '%data%scientist%')
     or (lower(v.name) like '%data%science%') 
     or (lower(v.name) like '%исследователь%данных%') 
     or (v.name like '%ML%' and v.name not like '%HTML%')
     or (lower(v.name) like '%machine%learning%')
     or (lower(v.name) like '%машинн%обучен%') 
     )
    and
    (
    (lower(v.name) like '%junior%') 
    or (v.experience = 'Нет опыта') 
    or (v.employment = 'Стажировка')
    )
'''

In [ ]:
# результат запроса
cnt_DS_junior = pd.read_sql_query(query_6_2, connection)
display('Подходящих вакансий для начинающего дата-сайентиста:', cnt_DS_junior.iloc[0][0])

'Подходящих вакансий для начинающего дата-сайентиста:'

51

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса
query_6_3 = f''' 
    select 
        count(v.id)
    from public.vacancies v  
    where (
        (lower(v.name) like '%data%scientist%')
        or (lower(v.name) like '%data%science%') 
        or (lower(v.name) like '%исследователь%данных%') 
        or (v.name like '%ML%' and v.name not like '%HTML%')
        or (lower(v.name) like '%machine%learning%')
        or (lower(v.name) like '%машинн%обучен%') 
        )
        and ((v.key_skills like '%SQL%') or (v.key_skills like '%postgres%'))
'''

In [ ]:
# результат запроса
cnt_DS_SQL_skill = pd.read_sql_query(query_6_3, connection)
display('Вакансий DS с навыком SQL:', cnt_DS_SQL_skill.iloc[0][0])

'Вакансий DS с навыком SQL:'

201

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса
query_6_4 = f''' 
    select 
        count(v.id)
    from public.vacancies v  
    where (
        (lower(v.name) like '%data%scientist%')
        or (lower(v.name) like '%data%science%') 
        or (lower(v.name) like '%исследователь%данных%') 
        or (v.name like '%ML%' and v.name not like '%HTML%')
        or (lower(v.name) like '%machine%learning%')
        or (lower(v.name) like '%машинн%обучен%') 
        )
        and (v.key_skills like '%Python%') 
    '''

In [ ]:
# результат запроса
cnt_DS_SQL_skill = pd.read_sql_query(query_6_4, connection)
display('Вакансий DS с навыком Python:', cnt_DS_SQL_skill.iloc[0][0])

'Вакансий DS с навыком Python:'

351

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
# текст запроса
query_6_5 = f''' 
    select 
        ROUND(avg(cardinality(regexp_split_to_array(v.key_skills, chr(9)))), 2)
    from public.vacancies v  
    where (
        (lower(v.name) like '%data%scientist%')
         or (lower(v.name) like '%data%science%') 
         or (lower(v.name) like '%исследователь%данных%') 
         or (v.name like '%ML%' and v.name not like '%HTML%')
         or (lower(v.name) like '%machine%learning%')
         or (lower(v.name) like '%машинн%обучен%') 
         )
    '''

In [ ]:
# результат запроса
avg_cnt_skills = pd.read_sql_query(query_6_5, connection)
display('среднее число ключевых навыков:', avg_cnt_skills.iloc[0][0])

'среднее число ключевых навыков:'

6.41

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
# текст запроса
query_6_6 = f''' 
select 
    v.experience "Опыт работы",
    round(avg((coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from)) / 2)) "Средняя зарплата"
from public.vacancies v  
where (
    (lower(v.name) like '%data%scientist%')
    or (lower(v.name) like '%data%science%') 
    or (lower(v.name) like '%исследователь%данных%') 
    or (v.name like '%ML%' and v.name not like '%HTML%')
    or (lower(v.name) like '%machine%learning%')
    or (lower(v.name) like '%машинн%обучен%'))
    and ((v.salary_from is not null) or (v.salary_to is not null))
group by v.experience
'''

In [ ]:
# результат запроса
avg_salary = pd.read_sql_query(query_6_6, connection)
avg_salary

,Опыт работы,Средняя зарплата
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

### Выводы:
Вакансии конкретно для DataSience занимают примерно 3,6% от общего числа вакансий. Причем для юниоров выделено всего 2,8% из этого числа вакансий. Трудно начать, но при анализе предлагаемых вилок зарплат и запрашиваемых ключевых навыков виден возможный быстрый карьерный рост (по уровню оплаты труда). Таким образом DS-специалисту важно иметь большой инструментарий для хорошего трудоустройства. 




# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

7. Рассмотрим топ-10 вакансий, имеющих отношение к данным

In [ ]:
query_7_1 = f'''
select 
    v.name "Название вакансии",
    count(v.id) "Количество"
from public.vacancies v  
where (lower(v.name) like '%data%') or (lower(v.name) like '%данн%') 
group by v.name
order by 2 desc
limit 10 
'''

In [ ]:
top_DS_vac = pd.read_sql_query(query_7_1, connection)
top_DS_vac

,Название вакансии,Количество
0,Аналитик данных,78
1,Data Engineer,63
2,Data Scientist,55
3,Data Analyst,51
4,Администратор баз данных,39
5,Разработчик баз данных,26
6,Senior Data Scientist,21
7,Senior Data Engineer,15
8,Data engineer,14
9,Middle Data Engineer,13


В топ-10 представлены три направления Data Data Analyst, Data Engineer и Data Scientist. Наиболее популярны аналитики и инженеры. Посмотрим как востребованы новички в этих вакансиях:

In [ ]:
query_7_2 = f''' 
select 
    v.name "Название вакансии",
    count(v.id) "Количество"
from public.vacancies v  
where ((lower(v.name) like '%data%') or (lower(v.name) like '%данн%') )
    and
    (
    (lower(v.name) like '%junior%') 
    or (v.experience = 'Нет опыта') 
    or (v.employment = 'Стажировка')
    )
group by v.name
order by 2 desc
limit 10
'''

In [ ]:
top_junior_vac = pd.read_sql_query(query_7_2, connection)
top_junior_vac

,Название вакансии,Количество
0,Junior Data Scientist (Валидация скоринговых м...,9
1,Администратор баз данных,8
2,Junior Data Engineer,7
3,Разработчик баз данных,4
4,Аналитик данных,4
5,Junior Data Scientist,3
6,Data Scientist/Machine Learning Engineer (Deve...,3
7,Data Analyst/Специалист по аналитике данных,3
8,Database administrator/architect,3
9,Администратор баз данных (стажер),3


Видим, что новичков больше готовы принять Data Scientist. Думаю, что это связанно DS имеет более широкий инструментарий 

7.2 Рассмотрим в каких регионах и сколько есть вакансий для новичков DS. Из пункта 6.2 мы знаем, что всего таких вакансий 51

In [ ]:
query_7_3 = f''' 
select 
    a.name "Район",
    count(v.id)
from public.vacancies v  
join public.areas a on v.area_id=a.id
where (
    (lower(v.name) like '%data%scientist%')
     or (lower(v.name) like '%data%science%') 
     or (lower(v.name) like '%исследователь%данных%') 
     or (v.name like '%ML%' and v.name not like '%HTML%')
     or (lower(v.name) like '%machine%learning%')
     or (lower(v.name) like '%машинн%обучен%') 
     )
    and
    (
    (lower(v.name) like '%junior%') 
    or (v.experience = 'Нет опыта') 
    or (v.employment = 'Стажировка')
    )
group by a.name
order by 2 desc, 1
'''

In [ ]:
area_juniorDS_vac = pd.read_sql_query(query_7_3, connection)
area_juniorDS_vac

,Район,count
0,Москва,19
1,Санкт-Петербург,9
2,Новосибирск,5
3,Воронеж,2
4,Екатеринбург,2
5,Казань,2
6,Минск,2
7,Пермь,2
8,Ростов-на-Дону,2
9,Томск,2


Видим, что распределение по районам аналогично распределению вакансий для опытных DS. Центрированно в Москве и СП, но и для нас сибиряков открываются вакансии в наших районах. Радует

In [ ]:
connection.close()

### Общие выводы:

В данном проекте были составлены многочисленных SQL-запросы позволившие изучить рынок вакансий. Проект хорошо показывает, возможности по извлечению нужной информации, хранящейся в базе данных, применяя инструментарий SQL.

В целом, если говорить про рынок вакансий, то видно, что наиболее высокая зарплата у DS, но войти в данную отрасль без опыта достаточно сложно поскольку для начинающих специалистов очень мало предложений, а для специалистов с опытом наоборот ощущается кадровый голод.

Так же мы видим, что предложения вакансий Data в основном сконцентрированы в самых больших городах, что удивительно, т.к. в возможностях данного направления имеется и удаленная работа и местоположение офисов вне крупных городов, что дает возможности трудоустройства специалистов без необходимости переезда и дополнительных затрат на дорогое жилье в крупных городах.
